In [1]:
import torch
from torch.nn import functional as F
from torch import nn

d_model = 128
block_size = 8
batch_size = 32
n_heads = 4
dropout = 0.1
learning_rate = 1e-3
epochs = 10_000
eval_iters = 200

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    torch.backends.cudnn.benchmark = True

In [2]:
class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, d_model,  dropout=0.1):

        super().__init__()
        assert d_model % n_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.n_heads = n_heads
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(d_model, d_model, bias=False)
        self.key = nn.Linear(d_model, d_model, bias=False)
        self.value = nn.Linear(d_model, d_model, bias=False)
        self.att_proj = nn.Linear(d_model, d_model, bias=False)
        self.register_buffer('mask', torch.triu(torch.ones(block_size, block_size), diagonal=1).bool())

    def forward(self, x):

        q = x
        k = x
        v = x
        B,T,_ = x.shape 
        dk = d_model // n_heads

        # linear projections
        q = self.query(q) 
        k = self.key(k) 
        v = self.value(v) 

        # add number of heads
        q = q.view(B,T,n_heads,dk).permute(0,2,1,3)   # B,T,h,dk
        k = k.view(B,T,n_heads,dk).permute(0,2,1,3)  
        v = v.view(B,T,n_heads,dk).permute(0,2,1,3)  
        
        # attention 

        x = q @ k.transpose(-2,-1) # B,h,T,dk @ B,h,dk,T --> B,h,T,T
        x = x * dk ** -0.5 # B,h,T,T
        x = x.masked_fill(self.mask, float('-inf')) # B,h,T,T
        x = F.softmax(x, dim=(-1)) # B,n_h,T,T 
        x = x @ v  # B,h,T,T @ B,T,h,dv --> B,h,T,dv
        B,h,T,dv = x.shape
        x = x.transpose(2,1).contiguous().view(B,T,h*dv) #B,T,C
        out = self.att_proj(x) # B,T,C

        return out

In [3]:
class Model(nn.Module):

    def __init__(self, vocab_size, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.embedding_table = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(block_size, d_model)
        self.mha = MultiHeadAttention(n_heads=n_heads, d_model=d_model)
        self.lnorm = nn.LayerNorm(d_model)
        self.out = nn.Linear(d_model, vocab_size)

    def forward(self, x, targets=None):

        embeds = self.embedding_table(x)
        positions = self.pos_embedding(torch.arange(block_size, device=device))
        x = embeds + positions

        x = self.mha(x)
        x = x + self.lnorm(x)
        logits = self.out(x)

        if targets == None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(input=logits, target=targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [4]:
data = open('text.txt').read()
vocab = list(sorted(set(data)))
vocab_size = len(vocab)

print(f'Vocab: {vocab[:10]}.')
print(f'Vocab size: {vocab_size}')

stoi = {c:i for i, c in enumerate(vocab)}
itos = {i:c for i, c in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[i] for i in i])

print(decode(encode(data[1100:1150])))
data = torch.tensor(encode(data))

n_tr = int(len(data) * 0.9)
n_val = len(data) - n_tr

train = data[:n_tr]
val = data[n_tr+1:]

print(f'Training samples: {train.shape[0]}')
print(f'Validation samples: {val.shape[0]}')

def make_batches(split):

    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

Xb, Yb = make_batches('train')

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3'].
Vocab size: 65
very dog to the commonalty.

Second Citizen:
Consi
Training samples: 1003854
Validation samples: 111539


In [5]:
m = Model(vocab_size).to(device)
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
n_params = sum(p.nelement() for p in m.parameters())
print(m)
print(f'Total parameters: {n_params}')

@torch.no_grad()
def estimate_loss(m):
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = make_batches(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

Model(
  (embedding_table): Embedding(65, 128)
  (pos_embedding): Embedding(8, 128)
  (mha): MultiHeadAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (query): Linear(in_features=128, out_features=128, bias=False)
    (key): Linear(in_features=128, out_features=128, bias=False)
    (value): Linear(in_features=128, out_features=128, bias=False)
    (att_proj): Linear(in_features=128, out_features=128, bias=False)
  )
  (lnorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (out): Linear(in_features=128, out_features=65, bias=True)
)
Total parameters: 83521


In [6]:
# Training
for epoch in range(epochs):

    Xb, Yb = make_batches('train')
    logits, loss = m(Xb, Yb) # B, C

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 999:
        l = estimate_loss(m)
        print(f"Iteration {epoch}. Training Loss: {l['train']:.3f}. Evaluation Loss: {l['val']:.3f}")


Iteration 999. Training Loss: 2.232. Evaluation Loss: 2.291
Iteration 1999. Training Loss: 2.166. Evaluation Loss: 2.241
Iteration 2999. Training Loss: 2.145. Evaluation Loss: 2.227
Iteration 3999. Training Loss: 2.129. Evaluation Loss: 2.214
Iteration 4999. Training Loss: 2.106. Evaluation Loss: 2.195
Iteration 5999. Training Loss: 2.076. Evaluation Loss: 2.191
Iteration 6999. Training Loss: 2.077. Evaluation Loss: 2.191
Iteration 7999. Training Loss: 2.075. Evaluation Loss: 2.188
Iteration 8999. Training Loss: 2.076. Evaluation Loss: 2.179
Iteration 9999. Training Loss: 2.053. Evaluation Loss: 2.147


In [8]:
context = torch.zeros(1, block_size).int().to(device)
predictions = m.generate(context, 500)
print(decode(predictions[0].tolist()))









This llow oud by Valow, an, here naig? mate of coffers
Bp of my to rne'llf to guonlifelf day how arve?
Now son a strefeay whedes nowits: it; this your drist.
The herime,
And might for gancoun, thriastit
you bofeth with therow heres he feerse not'cds lead's hing bestany for saminger rouu, sirsfoe llo slain lan sand will to caus ucher, from dan caby wang, and the good, th ean and to it your and queighth holity, wsir.

ROMPEY:
Je agphows, gain pe goitnt, whesir,
A bawild do thyseane.

EL:
I love fo
